## Project some statistical results on the freesurfer surfaces

In [53]:
# initiate some paths and files
rt=/Users/chris/Desktop/stats2surf
fslmaths ${rt}/epi.nii.gz -mas ${rt}/epi_mask.nii.gz ${rt}/epi_brain.nii.gz
cp ${rt}/epi_brain.nii.gz ${rt}/epi_brain_rescale.nii.gz


In [54]:
# create a 1mm (header) version of the epi
cp ${rt}/results/MeanAngle_th4_pv0.2.nii.gz ${rt}/results/MeanAngle_th4_pv0.2_rescale.nii.gz
cp ${rt}/results/tstat1.nii.gz ${rt}/results/tstat1_rescale.nii.gz
cp ${rt}/results/tstat2.nii.gz ${rt}/results/tstat2_rescale.nii.gz

3drefit -xdel 2.0 -ydel 2.0 -zdel 2.0 -keepcen ${rt}/epi_brain_rescale.nii.gz
3drefit -xdel 2.0 -ydel 2.0 -zdel 2.0 -keepcen ${rt}/results/MeanAngle_th4_pv0.2_rescale.nii.gz
3drefit -xdel 2.0 -ydel 2.0 -zdel 2.0 -keepcen ${rt}/results/tstat1_rescale.nii.gz
3drefit -xdel 2.0 -ydel 2.0 -zdel 2.0 -keepcen ${rt}/results/tstat2_rescale.nii.gz

++ 3drefit: AFNI version=AFNI_17.2.08 (Aug 18 2017) [64-bit]
++ Authored by: RW Cox
++ Processing AFNI dataset /Users/chris/Desktop/stats2surf/epi_brain_rescale.nii.gz
 + loading and re-writing dataset /Users/chris/Desktop/stats2surf/epi_brain_rescale.nii.gz (/Users/chris/Desktop/stats2surf/epi_brain_rescale.nii.gz in NIFTI storage)
++ 3drefit processed 1 datasets
++ 3drefit: AFNI version=AFNI_17.2.08 (Aug 18 2017) [64-bit]
++ Authored by: RW Cox
++ Processing AFNI dataset /Users/chris/Desktop/stats2surf/results/MeanAngle_th4_pv0.2_rescale.nii.gz
 + loading and re-writing dataset /Users/chris/Desktop/stats2surf/results/MeanAngle_th4_pv0.2_rescale.nii.gz (/Users/chris/Desktop/stats2surf/results/MeanAngle_th4_pv0.2_rescale.nii.gz in NIFTI storage)
++ 3drefit processed 1 datasets
++ 3drefit: AFNI version=AFNI_17.2.08 (Aug 18 2017) [64-bit]
++ Authored by: RW Cox
++ Processing AFNI dataset /Users/chris/Desktop/stats2surf/results/tstat1_rescale.nii.gz
 + loading and re-writing dataset /User

In [34]:
# do a flirt registration on the orgiginal files
mkdir -p ${rt}/flirt
#lirt -ref ${rt}/fsSurf/src/brain_org.nii.gz \
#    -wmseg ${rt}/fsSurf/src/wm_org.nii.gz \
#    -in ${rt}/epi_brain.nii.gz \
#    -out ${rt}/flirt/epi2anat.nii.gz \
#    -omat ${rt}/flirt/epi2anat.mat \
#    -pedir -2      

In [35]:
# do a flirt reistration on the 1mm voxel-header files
flirt -ref ${rt}/fsSurf/src/brain.nii.gz \
    -wmseg ${rt}/fsSurf/src/wm.nii.gz \
    -in ${rt}/epi_brain_rescale.nii.gz \
    -out ${rt}/flirt/epi2anat_rescale.nii.gz \
    -omat ${rt}/flirt/epi2anat_rescale.mat \
    -pedir -2  

In [41]:
# tkregister
# convert flirt registration matrix to tkregister (and check registration)
tkregister2 --mov ${rt}/epi_brain_rescale.nii.gz \
    --targ ${rt}/fsSurf/src/brain.nii.gz \
    --fsl ${rt}/flirt/epi2anat_rescale.mat \
    --reg ${rt}/reg.fsl.dat \
    --s Danny

tkregister_tcl /Users/chris/Documents/MRI/freesurfer/tktools/tkregister2.tcl
---- FSL registration matrix --------
 2.03386  -0.01328   0.00541   1.95302;
 0.01977   2.00845   0.17626  -2.94179;
-0.01286   0.00636   2.05322   3.09676;
 0.00000   0.00000   0.00000   1.00000;
---------------------------------------
target  volume /Users/chris/Desktop/stats2surf/fsSurf/src/brain.nii.gz
movable volume /Users/chris/Desktop/stats2surf/epi_brain_rescale.nii.gz
reg file       /Users/chris/Desktop/stats2surf/reg.fsl.dat
LoadVol        1
ZeroCRAS       0
$Id: tkregister2.c,v 1.129 2014/02/26 21:31:58 greve Exp $
Diagnostic Level -1
INFO: loading target /Users/chris/Desktop/stats2surf/fsSurf/src/brain.nii.gz
INFO: target does not conform to COR format, so I'm going to
reslice to COR. This will not affect the final registration.
Ttarg: --------------------
-1.00000   0.00000   0.00000   128.00000;
 0.00000   0.00000   1.00000  -128.00000;
 0.00000  -1.00000   0.00000   128.00000;
 0.00000   0.0000

In [55]:
mri_vol2surf --trgsubject Danny \
    --src ${rt}/results/tstat1_rescale.nii.gz \
    --out ${rt}/results/targ1.w --out_type paint \
    --srcreg ${rt}/reg.fsl.dat \
    --hemi lh
mri_vol2surf --trgsubject Danny \
    --src ${rt}/results/tstat2_rescale.nii.gz \
    --out ${rt}/results/targ2.w --out_type paint \
    --srcreg ${rt}/reg.fsl.dat \
    --hemi lh    
mri_vol2surf --trgsubject Danny \
    --src ${rt}/results/MeanAngle_th4_pv0.2_rescale.nii.gz \
    --out ${rt}/results/angle_map.w --out_type paint \
    --srcreg ${rt}/reg.fsl.dat \
    --hemi lh

INFO: output format is paint
srcvol = /Users/chris/Desktop/stats2surf/results/tstat1_rescale.nii.gz
srcreg = /Users/chris/Desktop/stats2surf/reg.fsl.dat
srcregold = 0
srcwarp unspecified
surf = white
hemi = lh
trgsubject = Danny
surfreg = sphere.reg
reshape = 0
interp = nearest
float2int = round
GetProjMax = 0
INFO: float2int code = 0
Done loading volume
INFO: This REGISTER_DAT transform is valid only for volumes between  COR types with c_(r,a,s) = 0.
Input reg is register.dat
         from that listed in the registration file (1,1,1)
-------- original matrix -----------
-0.49163   0.00158   0.00326   1.99257;
 0.00309   0.48716   0.00152   0.09261;
 0.00511   0.04274   0.49800   1.61333;
 0.00000   0.00000   0.00000   1.00000;
-------- original matrix -----------
Reading surface /Users/chris/Documents/MRI/freesurfer/subjects//Danny/surf/lh.white
Done reading source surface
Mapping Source Volume onto Source Subject Surface
 1 0 0 0
using old
Done mapping volume to surface
Number of sou

In [57]:
# vol2surf
tksurfer Danny lh graymid \
-patch full.patch.flat \
-overlay ${rt}/results/targ2.w \
-overlay-reg ${rt}/reg.fsl.dat 

displaying patch full.patch.flat...
subject is Danny
hemi    is lh
surface is graymid
surfer: current subjects dir: /Users/chris/Documents/MRI/freesurfer/subjects/
surfer: not in "scripts" dir ==> using cwd for session root
surfer: session root data dir ($session) set to:
surfer:     /Users/chris/Documents/MRI_ANALYSIS/NHP-Freesurfer/Notebooks
checking for nofix files in 'graymid'
Reading image info (/Users/chris/Documents/MRI/freesurfer/subjects//Danny)
Reading /Users/chris/Documents/MRI/freesurfer/subjects//Danny/mri/T1.mgz
surfer: Reading header info from /Users/chris/Documents/MRI/freesurfer/subjects//Danny/mri/T1.mgz
ltMNIreadEx: could not open file /Users/chris/Documents/MRI/freesurfer/subjects//Danny/mri/transforms/talairach.xfm
No such file or directory
surfer: Talairach xform file not found (ignored)
surfer: vertices=75586, faces=151172
         Talairach coords will be incorrect.
surfer: curvature read: min=-1.147546 max=0.665777
surfer: single buffered window
surfer: tkoInit

In [ ]:
tkregister2 --sd ${targ_dir} --s ${structural_session} --targ ${targ_dir}/${targ_vol} --mov ${mov_dir}/${mov_vol} --reg ${out_reg_dir}/${out_reg} --regheader --movscale 2 ${tkregister2_opts}#--noedit
